In [2]:
!pip install livelossplot
from keras.models import Model, Sequential
from keras.layers import Input, BatchNormalization, Conv2D, ConvLSTM2D, concatenate

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy.ma as ma
import os
import joblib
from livelossplot import PlotLossesKeras

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
input_shape_one = 6

In [4]:
def conv_lstm_model():
    cmorph_in = Input(shape = (input_shape_one, 285, 245, 1))
    cmorph_2 = ConvLSTM2D(filters=50, kernel_size=(3, 3),
                         padding='same', return_sequences=False,
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False)(cmorph_in)
    cmorph_out = BatchNormalization()(cmorph_2)
    
    
    imerg_in = Input(shape = (input_shape_one, 285, 245, 1))
    imerg_2 = ConvLSTM2D(filters=50, kernel_size=(3, 3),
                         padding='same', return_sequences=False,
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False)(imerg_in)
    imerg_out = BatchNormalization()(imerg_2)
    
    
    tamsat_in = Input(shape = (input_shape_one, 285, 245, 1))
    tamsat_2 = ConvLSTM2D(filters=50, kernel_size=(3, 3),
                         padding='same', return_sequences=False,
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False)(tamsat_in)
    tamsat_out = BatchNormalization()(tamsat_2)
    
    
    chirps_in = Input(shape = (input_shape_one, 285, 245, 1))
    chirps_2 = ConvLSTM2D(filters=50, kernel_size=(3, 3),
                         padding='same', return_sequences=False,
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False)(chirps_in)
    chirps_out = BatchNormalization()(chirps_2)
    
    
    merge_1 = concatenate([cmorph_out, imerg_out, tamsat_out, chirps_out])
    merge = BatchNormalization()(merge_1)
    
    out = Conv2D(filters=1, kernel_size=(1, 1),
                   activation='relu',
                   padding='same', data_format='channels_last')(merge)
    
    
    model = Model(inputs = [cmorph_in, imerg_in, tamsat_in, chirps_in], outputs = [out])
    print(model.summary())
    return model

In [ ]:
print(conv_lstm_model())

In [ ]:
def stacked_2_model():   
    model = Sequential(name='model_stacked_2')
    model.add(ConvLSTM2D(filters=50, kernel_size=(3, 3),
                         input_shape=(input_shape_one, 285, 245, 4), padding='same', return_sequences=True, 
                         activation='tanh', recurrent_activation='hard_sigmoid',
                         kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False ))
    model.add(BatchNormalization())
    
    model.add(ConvLSTM2D(filters=50, kernel_size=(3, 3), padding='same', return_sequences=False, 
                         activation='tanh', recurrent_activation='hard_sigmoid', 
                         kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                         dropout=0.1, recurrent_dropout=0.1, go_backwards=False ))
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=1, kernel_size=(1, 1),
                   activation='relu',
                   padding='same', data_format='channels_last'))
    print(model.summary())
    
    return model

**Load labels and predictors**

In [25]:
labels = np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/labels/labels_all.npy')
labels_train = labels[5:1461]
labels_val = labels[1466:]

In [6]:
cmorph = np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/cmorph_feature_tensor.npy')
#imerg = np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/imerg_feature_tensor.npy')
#tamsat = np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/tamsat_feature_tensor.npy')
#chirps = np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/chirps_feature_tensor.npy')

In [36]:
cmorph_train = cmorph[:1456]
cmorph_val = cmorph[1461:]
imerg_train = imerg[:1456]
imerg_val = imerg[1461:]
tamsat_train = tamsat[:1456]
tamsat_val = tamsat[1461:]
chirps_train = chirps[:1456]
chirps_val = chirps[1461:]

In [1]:
cmorph_train.shape

NameError: ignored

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss = 'mean_squared_error', optimizer = opt, metrics = 'mse')

mc = ModelCheckpoint('model_stacked_1.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

history = conv_lstm_model.fit(x = [cmorph_train, imerg_train, tamsat_train, chirps_train],
                    y = labels_train, validation_data = ([cmorph_val, imerg_val, tamsat_val, chirps_val], labels_val),
                    epochs = 250, batch_size = 16, verbose = 2, callbacks=[es, mc, PlotLossesKeras()], shuffle = False)
np.save('my_history.npy',history.history)